In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 

In [2]:
import logistic_regression
import reduceClasses
import binaryRatio
import train_with_dir
import test 
import get_confidence_interval

In [3]:
NUM_CLASSES = 10
NUM_CLASSES_REDUCED = 2
n_epochs = 20
batch_size_train = 64
batch_size_test = 1000
momentum = 0.5

ratio = (100, 1)

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
learning_rates = [0.00001, 0.1, 0.01, 0.001, 0.0001]
learning_rate_strings = ["0.00001", "0.1", "0.01", "0.001", "0.0001"]

In [5]:
train_mnist = torchvision.datasets.MNIST('mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor() ]))
                             #  torchvision.transforms.Normalize(
                           #      (0.1307,), (0.3081,))
                             


test_mnist = torchvision.datasets.MNIST('mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor() ]))
                             #  torchvision.transforms.Normalize(
                              #   (0.1307,), (0.3081,))])
                             
                            

reduced_train_mnist = reduceClasses.Reduce(train_mnist, NUM_CLASSES_REDUCED, nums=(6, 8))
reduced_test_mnist = reduceClasses.Reduce(test_mnist, NUM_CLASSES_REDUCED, nums=(6, 8))


reduced_train_mnist_ratio = binaryRatio.Ratio(train_mnist, 2, ratio, nums=(6, 8)) 

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [6]:
train_loader_reduced = DataLoader(reduced_train_mnist, batch_size=batch_size_train, shuffle=False) 
train_loader_reduced_ratio = DataLoader(reduced_train_mnist_ratio, batch_size=batch_size_train, shuffle=False)
train_loader_normal = DataLoader(train_mnist, batch_size=batch_size_train, shuffle=False)

test_loader_reduced = DataLoader(reduced_test_mnist, batch_size=batch_size_test, shuffle=False) 
test_loader_normal = DataLoader(test_mnist, batch_size=batch_size_test, shuffle=False)

In [7]:
for lr_s, learning_rate in zip(learning_rate_strings,learning_rates):
    for i in range(4): 
        network = logistic_regression.Net(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                          momentum=momentum)

        for epoch in range(n_epochs):
                _, _ = train_with_dir.train(epoch, train_loader_reduced, network, optimizer, f'learning_rate_results/lr{lr_s}/reduced{i}/model{epoch}', loss_fn=nn.BCELoss())
                _, _, _ = test.test(test_loader_reduced, network,loss_fn=nn.BCELoss())

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

Train Epoch: 0 [0/11769 (0%)]	Loss: 18.650421
Train Epoch: 0 [640/11769 (5%)]	Loss: 17.904667
Train Epoch: 0 [1280/11769 (11%)]	Loss: 5.128709
Train Epoch: 0 [1920/11769 (16%)]	Loss: 2.545918
Train Epoch: 0 [2560/11769 (22%)]	Loss: 5.769418
Train Epoch: 0 [3200/11769 (27%)]	Loss: 0.925031
Train Epoch: 0 [3840/11769 (33%)]	Loss: 0.801888
Train Epoch: 0 [4480/11769 (38%)]	Loss: 11.091557
Train Epoch: 0 [5120/11769 (43%)]	Loss: 5.124515
Train Epoch: 0 [5760/11769 (49%)]	Loss: 0.530866
Train Epoch: 0 [6400/11769 (54%)]	Loss: 2.441534
Train Epoch: 0 [7040/11769 (60%)]	Loss: 0.558646
Train Epoch: 0 [7680/11769 (65%)]	Loss: 0.356838
Train Epoch: 0 [8320/11769 (71%)]	Loss: 0.252352
Train Epoch: 0 [8960/11769 (76%)]	Loss: 5.439730
Train Epoch: 0 [9600/11769 (82%)]	Loss: 0.498284
Train Epoch: 0 [10240/11769 (87%)]	Loss: 0.308169
Train Epoch: 0 [10880/11769 (92%)]	Loss: 1.590908
Train Epoch: 0 [11520/11769 (98%)]	Loss: 0.000076

Test set: Avg. loss: 0.0014040692869427289, AUC: 0.9846413858440539 

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde


Test set: Avg. loss: 0.001070423834565757, AUC: 0.9891002173419128 

Train Epoch: 2 [0/11769 (0%)]	Loss: 0.001120
Train Epoch: 2 [640/11769 (5%)]	Loss: 3.401111
Train Epoch: 2 [1280/11769 (11%)]	Loss: 0.021299
Train Epoch: 2 [1920/11769 (16%)]	Loss: 1.640905
Train Epoch: 2 [2560/11769 (22%)]	Loss: 0.042120
Train Epoch: 2 [3200/11769 (27%)]	Loss: 1.562502
Train Epoch: 2 [3840/11769 (33%)]	Loss: 1.834632
Train Epoch: 2 [4480/11769 (38%)]	Loss: 5.503201
Train Epoch: 2 [5120/11769 (43%)]	Loss: 1.824020
Train Epoch: 2 [5760/11769 (49%)]	Loss: 0.025208
Train Epoch: 2 [6400/11769 (54%)]	Loss: 0.496279
Train Epoch: 2 [7040/11769 (60%)]	Loss: 0.016963
Train Epoch: 2 [7680/11769 (65%)]	Loss: 0.265571
Train Epoch: 2 [8320/11769 (71%)]	Loss: 0.012477
Train Epoch: 2 [8960/11769 (76%)]	Loss: 1.129835
Train Epoch: 2 [9600/11769 (82%)]	Loss: 0.439133
Train Epoch: 2 [10240/11769 (87%)]	Loss: 0.077463
Train Epoch: 2 [10880/11769 (92%)]	Loss: 1.562520
Train Epoch: 2 [11520/11769 (98%)]	Loss: 0.000130

T

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

...

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

Train Epoch: 15 [3200/11769 (27%)]	Loss: 1.570538
Train Epoch: 15 [3840/11769 (33%)]	Loss: 0.000528
Train Epoch: 15 [4480/11769 (38%)]	Loss: 0.000000
Train Epoch: 15 [5120/11769 (43%)]	Loss: 1.562567
Train Epoch: 15 [5760/11769 (49%)]	Loss: 0.000017
Train Epoch: 15 [6400/11769 (54%)]	Loss: 0.005244
Train Epoch: 15 [7040/11769 (60%)]	Loss: 0.000000
Train Epoch: 15 [7680/11769 (65%)]	Loss: 0.524039
Train Epoch: 15 [8320/11769 (71%)]	Loss: 0.010227
Train Epoch: 15 [8960/11769 (76%)]	Loss: 0.000007
Train Epoch: 15 [9600/11769 (82%)]	Loss: 0.000003
Train Epoch: 15 [10240/11769 (87%)]	Loss: 0.000000
Train Epoch: 15 [10880/11769 (92%)]	Loss: 0.000166
Train Epoch: 15 [11520/11769 (98%)]	Loss: 0.000000

Test set: Avg. loss: 0.00043400218922405755, AUC: 0.9966032288348845 

Train Epoch: 16 [0/11769 (0%)]	Loss: 0.000000
Train Epoch: 16 [640/11769 (5%)]	Loss: 1.562500
Train Epoch: 16 [1280/11769 (11%)]	Loss: 0.014173
Train Epoch: 16 [1920/11769 (16%)]	Loss: 1.562717
Train Epoch: 16 [2560/11769 (22

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

Train Epoch: 17 [640/11769 (5%)]	Loss: 1.562500
Train Epoch: 17 [1280/11769 (11%)]	Loss: 0.003883
Train Epoch: 17 [1920/11769 (16%)]	Loss: 1.563251
Train Epoch: 17 [2560/11769 (22%)]	Loss: 0.000000
Train Epoch: 17 [3200/11769 (27%)]	Loss: 1.585555
Train Epoch: 17 [3840/11769 (33%)]	Loss: 0.001088
Train Epoch: 17 [4480/11769 (38%)]	Loss: 0.000004
Train Epoch: 17 [5120/11769 (43%)]	Loss: 1.563074
Train Epoch: 17 [5760/11769 (49%)]	Loss: 0.000086
Train Epoch: 17 [6400/11769 (54%)]	Loss: 0.000016
Train Epoch: 17 [7040/11769 (60%)]	Loss: 0.000006
Train Epoch: 17 [7680/11769 (65%)]	Loss: 0.560719
Train Epoch: 17 [8320/11769 (71%)]	Loss: 0.002218
Train Epoch: 17 [8960/11769 (76%)]	Loss: 0.000219
Train Epoch: 17 [9600/11769 (82%)]	Loss: 0.000039
Train Epoch: 17 [10240/11769 (87%)]	Loss: 0.000000
Train Epoch: 17 [10880/11769 (92%)]	Loss: 0.000328
Train Epoch: 17 [11520/11769 (98%)]	Loss: 0.000000

Test set: Avg. loss: 0.00039460589457496106, AUC: 0.9971192551216815 

Train Epoch: 18 [0/11769 (0

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

In [8]:
counter = 0; 
for learning_rate in learning_rates:
    lr_s = learning_rate_strings[counter]
    counter += 1
    for i in range(4): 
        network = logistic_regression.Net(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                          momentum=momentum)

        for epoch in range(n_epochs):
                _, _ = train_with_dir.train(epoch, train_loader_reduced_ratio, network, optimizer, f'learning_rate_results/lr{lr_s}/reduced_ratio{i}/model{epoch}', loss_fn=nn.BCELoss())
                _, _, _ = test.test(test_loader_reduced, network, loss_fn=nn.BCELoss())

Train Epoch: 0 [0/5977 (0%)]	Loss: 22.863739
Train Epoch: 0 [640/5977 (11%)]	Loss: 1.563077
Train Epoch: 0 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 0 [1920/5977 (32%)]	Loss: 0.061230
Train Epoch: 0 [2560/5977 (43%)]	Loss: 0.000535
Train Epoch: 0 [3200/5977 (53%)]	Loss: 0.000154
Train Epoch: 0 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 0 [4480/5977 (74%)]	Loss: 0.151027
Train Epoch: 0 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 0 [5760/5977 (96%)]	Loss: 0.000000

Test set: Avg. loss: 0.02231079796579807, AUC: 0.7548778684202629 

Train Epoch: 1 [0/5977 (0%)]	Loss: 0.000001
Train Epoch: 1 [640/5977 (11%)]	Loss: 0.113619
Train Epoch: 1 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 1 [1920/5977 (32%)]	Loss: 0.028364
Train Epoch: 1 [2560/5977 (43%)]	Loss: 0.000551
Train Epoch: 1 [3200/5977 (53%)]	Loss: 0.000004
Train Epoch: 1 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 1 [4480/5977 (74%)]	Loss: 0.166951
Train Epoch: 1 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 1 [5760/5977 (9

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde


Test set: Avg. loss: 0.01593204265302259, AUC: 0.8259903632224905 

Train Epoch: 2 [0/5977 (0%)]	Loss: 0.000002
Train Epoch: 2 [640/5977 (11%)]	Loss: 0.158728
Train Epoch: 2 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 2 [1920/5977 (32%)]	Loss: 0.100897
Train Epoch: 2 [2560/5977 (43%)]	Loss: 0.071282
Train Epoch: 2 [3200/5977 (53%)]	Loss: 0.000002
Train Epoch: 2 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 2 [4480/5977 (74%)]	Loss: 0.249104
Train Epoch: 2 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 2 [5760/5977 (96%)]	Loss: 0.000000

Test set: Avg. loss: 0.008926409865511624, AUC: 0.8926392038512815 

Train Epoch: 3 [0/5977 (0%)]	Loss: 0.000002
Train Epoch: 3 [640/5977 (11%)]	Loss: 0.165113
Train Epoch: 3 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 3 [1920/5977 (32%)]	Loss: 0.194467
Train Epoch: 3 [2560/5977 (43%)]	Loss: 0.218695
Train Epoch: 3 [3200/5977 (53%)]	Loss: 0.000001
Train Epoch: 3 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 3 [4480/5977 (74%)]	Loss: 1.569655
Train E

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

Train Epoch: 4 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 4 [4480/5977 (74%)]	Loss: 1.563611
Train Epoch: 4 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 4 [5760/5977 (96%)]	Loss: 0.000000

Test set: Avg. loss: 0.005992629266426923, AUC: 0.9468744775434792 

Train Epoch: 5 [0/5977 (0%)]	Loss: 0.000000
Train Epoch: 5 [640/5977 (11%)]	Loss: 0.000362
Train Epoch: 5 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 5 [1920/5977 (32%)]	Loss: 0.168047
Train Epoch: 5 [2560/5977 (43%)]	Loss: 0.221104
Train Epoch: 5 [3200/5977 (53%)]	Loss: 0.000000
Train Epoch: 5 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 5 [4480/5977 (74%)]	Loss: 1.562761
Train Epoch: 5 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 5 [5760/5977 (96%)]	Loss: 0.000000

Test set: Avg. loss: 0.005318501968067872, AUC: 0.9554213303725677 

Train Epoch: 6 [0/5977 (0%)]	Loss: 0.000000
Train Epoch: 6 [640/5977 (11%)]	Loss: 0.000200
Train Epoch: 6 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 6 [1920/5977 (32%)]	Loss: 0.161001
Train 

...

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde


Test set: Avg. loss: 0.0032843155150087725, AUC: 0.9899559743305055 

Train Epoch: 13 [0/5977 (0%)]	Loss: 0.000000
Train Epoch: 13 [640/5977 (11%)]	Loss: 0.000005
Train Epoch: 13 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 13 [1920/5977 (32%)]	Loss: 0.000470
Train Epoch: 13 [2560/5977 (43%)]	Loss: 0.001865
Train Epoch: 13 [3200/5977 (53%)]	Loss: 0.000000
Train Epoch: 13 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 13 [4480/5977 (74%)]	Loss: 0.000402
Train Epoch: 13 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 13 [5760/5977 (96%)]	Loss: 0.000000

Test set: Avg. loss: 0.0036975482235784116, AUC: 0.988211237477119 

Train Epoch: 14 [0/5977 (0%)]	Loss: 0.000000
Train Epoch: 14 [640/5977 (11%)]	Loss: 0.000000
Train Epoch: 14 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 14 [1920/5977 (32%)]	Loss: 0.000018
Train Epoch: 14 [2560/5977 (43%)]	Loss: 0.000039
Train Epoch: 14 [3200/5977 (53%)]	Loss: 0.000000
Train Epoch: 14 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 14 [4480/5977 (74%)]	Lo

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde


Test set: Avg. loss: 0.0037463941682693135, AUC: 0.9877552267086203 

Train Epoch: 16 [0/5977 (0%)]	Loss: 0.000000
Train Epoch: 16 [640/5977 (11%)]	Loss: 0.000000
Train Epoch: 16 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 16 [1920/5977 (32%)]	Loss: 0.000013
Train Epoch: 16 [2560/5977 (43%)]	Loss: 0.000028
Train Epoch: 16 [3200/5977 (53%)]	Loss: 0.000000
Train Epoch: 16 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 16 [4480/5977 (74%)]	Loss: 0.000144
Train Epoch: 16 [5120/5977 (85%)]	Loss: 0.000000
Train Epoch: 16 [5760/5977 (96%)]	Loss: 0.000000

Test set: Avg. loss: 0.00376081491355817, AUC: 0.9877621927955657 

Train Epoch: 17 [0/5977 (0%)]	Loss: 0.000000
Train Epoch: 17 [640/5977 (11%)]	Loss: 0.000000
Train Epoch: 17 [1280/5977 (21%)]	Loss: 0.000000
Train Epoch: 17 [1920/5977 (32%)]	Loss: 0.000012
Train Epoch: 17 [2560/5977 (43%)]	Loss: 0.000026
Train Epoch: 17 [3200/5977 (53%)]	Loss: 0.000000
Train Epoch: 17 [3840/5977 (64%)]	Loss: 0.000000
Train Epoch: 17 [4480/5977 (74%)]	Los

/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
/Users/caralee/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=orde

In [9]:
for lr in range(len(learning_rates) - 1):
    accuracies = []
    for i in range(4):
        model_accuracies = []
        network = logistic_regression.Net(NUM_CLASSES_REDUCED)
        _, y_preds, y_true = test.test(test_loader_reduced, network) 
        _, _, acc = get_confidence_interval.get_confidence_interval(y_preds, y_true)
        model_accuracies.append(acc)
        for epoch in range(n_epochs):
            state_dict = torch.load(f'learning_rate_results/lr{learning_rate_strings[lr]}/reduced_ratio{i}/model{epoch}')
            network.load_state_dict(state_dict)
            _, y_preds, y_true = test.test(test_loader_reduced, network) 
            _, _, acc = get_confidence_interval.get_confidence_interval(y_preds, y_true)
            model_accuracies.append(acc)
        accuracies.append(model_accuracies)


    for j in range(4):
        plt.plot(np.arange(-1, n_epochs), accuracies[j])
        plt.title("Logistic regression 100:1 lr=" + learning_rate_strings[lr])
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend() 
    plt.show()

RuntimeError: expected scalar type Long but found Float

In [ ]:
for lr in range(len(learning_rates) - 1):
    accuracies = []
    for i in range(4):
        model_accuracies = []
        network = logistic_regression.Net(NUM_CLASSES_REDUCED)
        _, y_preds, y_true = test.test(test_loader_reduced, network) 
        _, _, acc = get_confidence_interval.get_confidence_interval(y_preds, y_true)
        model_accuracies.append(acc)
        for epoch in range(n_epochs):
            state_dict = torch.load(f'learning_rate_results/lr{learning_rate_strings[lr]}/reduced{i}/model{epoch}')
            network.load_state_dict(state_dict)
            _, y_preds, y_true = test.test(test_loader_reduced, network) 
            _, _, acc = get_confidence_interval.get_confidence_interval(y_preds, y_true)
            model_accuracies.append(acc)
        accuracies.append(model_accuracies)


    for j in range(4):
        plt.plot(np.arange(-1, n_epochs), accuracies[j])
        plt.title("Logistic regression lr=" + learning_rate_strings[lr])
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend() 
    plt.show()